In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('users.csv')
df

,name,username,password,gender,email,phone,address,create_at,status
0,Phạm Hướng Dương,/)_J^,Rj\Qvyq{fa,0,phamduong@hotmail.com,927146398,"Phường Tân Hòa, Thành phố Biên Hòa, Tỉnh Đồng Nai",2024-06-04,banned
1,Trần An Cơ,8t<6iC,"X,R$MJ""5",1,co_570903@gmail.com,795211838,"Xã Minh Tân, Huyện Cẩm Khê, Tỉnh Phú Thọ",2024-08-06,banned
2,Phạm Minh Thu,"3pJSYF""!i",kp)KyWQp,0,thupham@stu.ptit.edu.vn,973433767,"Phường Vạn Thạnh, Thành phố Nha Trang, Tỉnh Kh...",2024-10-03,inactive
3,Huỳnh Hàm Ý,yu`/6'},Ln\Ke]_Q,0,huynhy@icloud.com,321942138,"Xã Bản Lầm, Huyện Thuận Châu, Tỉnh Sơn La",2023-12-23,inactive
4,Hoàng Phương Thi,"[@{+""#4",1b?l|\izKq,0,hoang.thi758@stu.ptit.edu.vn,799188866,"Phường Hòa Hương, Thành phố Tam Kỳ, Tỉnh Quảng...",2025-02-07,banned
...,...,...,...,...,...,...,...,...,...
999995,Trần Tường Lâm,<e2(z8,*TLpc\@=k2=3,1,tran.lam@gmail.com,903615342,"Phường 2, Thị xã Quảng Trị, Tỉnh Quảng Trị",2025-01-20,banned
999996,Ngô Thiên Khánh,dg|oGt!,iGeyWaxD,0,khanh_290@gdscptit.dev,962998193,"Phường Yên Giang, Thị xã Quảng Yên, Tỉnh Quảng...",2024-08-19,banned
999997,Phạm Mai Loan,9JHjLb,{R~h5sC.`,0,loan.pham@yahoo.com,845000454,"Xã Kim Lư, Huyện Na Rì, Tỉnh Bắc Kạn",2024-02-10,active
999998,Dương Kiều Thu,JMzQf8PW,"k\6`""z}j3",0,thu_428@icloud.com,350636152,"Xã Quí Quân, Huyện Yên Sơn, Tỉnh Tuyên Quang",2023-04-19,active


In [13]:
import requests
from bs4 import BeautifulSoup
import re
import json

url = 'https://www.thegioididong.com/aj/CategoryV5/Product/'
comment_api = 'https://www.thegioididong.com/aj/ProductV4/RatingCommentList/'
spec = 'https://www.thegioididong.com/aj/ProductV4/GetFullSpec/'
domain = 'https://www.thegioididong.com'

outcomment = open('comment', 'w')
specfile = open('spec', 'w')
page_idx =-1 
while True: 
    page_idx += 1
    form = {"Category":42, "Manufacture":0, "PriceRange":0, "Feature":0, "Property":0,"OrderBy":0,"PageSize":30,"PageIndex":page_idx,"ClearCache":0}
    page_text = requests.post(url,params=form).text
    if "Không tìm thấy" in page_text: break
    soup = BeautifulSoup(page_text, 'lxml')

    for link in soup.find_all('a'):
        product_url = link.get('href')
        product_name = link.find('h3')
        if product_name != None:
            product_name = product_name.text
            print(product_name)
            product_page = requests.post(domain+product_url).text
            product_soup = BeautifulSoup(product_page, 'lxml')
            try:
                product_description = product_soup.find('article', {'class':'area_article'}).text
            except:
                print('product invalid')
                continue
            m = re.search('var productID = \'(.+)\'[,;]{1}', product_page)
            if m:
                product_id = int(m.group(1))

                spec_params = {'productID':product_id}                
                spec_page = requests.post('https://www.thegioididong.com/aj/ProductV4/GetFullSpec/', params=spec_params).text
                spec_json = json.loads(spec_page)
                spec_soup = BeautifulSoup(spec_json['spec'], 'lxml')
                spec_tags = spec_soup.find_all('li')
                spec_dict = {}
                spec_dict['productID'] = product_id
                spec_dict['description'] = product_description
                key, value = '', ''
                for tag in spec_tags:    
                    if 'label' in str(tag):
                        key = tag.text
                    else:
                        value = tag.find('span').text + ':' + tag.find('div').text
                        spec_dict[key] = spec_dict.get(key, '') + ' . '+ value

                specfile.write(json.dumps(spec_dict)+'\n')

                cmt_page = 0
                comment_text = "init"
                while comment_text != "":
                    cmt_page += 1
                    comment_form = {"productid":product_id, "page":cmt_page, "score":0};
                    comment_text = requests.post(comment_api, params=comment_form).text
                    comment_soup = BeautifulSoup(comment_text, 'lxml')
                    for cmt in comment_soup.select('div.rc > p'):
                        data = cmt.select('i')
                        stars = [r for r in data if 'iconcom-txtstar' in str(r)]
                        nstar = len(stars)
                        user_cmt = data[-1].text
                        
                        features = [product_name, product_id, nstar, user_cmt]
                        features = map(str, features)
                        features = [re.sub('\n|\|', ' ', feature) for feature in features]
                        features = [re.sub('\s\s+', ' ', feature) for feature in features]

                        strcomment = '|'.join(features)
                        outcomment.write(strcomment+'\n')

outcomment.close()                       
specfile.close()

<>:71: SyntaxWarning: invalid escape sequence '\|'
<>:72: SyntaxWarning: invalid escape sequence '\s'
<>:71: SyntaxWarning: invalid escape sequence '\|'
<>:72: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21012\3724973599.py:71: SyntaxWarning: invalid escape sequence '\|'
  features = [re.sub('\n|\|', ' ', feature) for feature in features]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21012\3724973599.py:72: SyntaxWarning: invalid escape sequence '\s'
  features = [re.sub('\s\s+', ' ', feature) for feature in features]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21012\3724973599.py:71: SyntaxWarning: invalid escape sequence '\|'
  features = [re.sub('\n|\|', ' ', feature) for feature in features]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_21012\3724973599.py:72: SyntaxWarning: invalid escape sequence '\s'
  features = [re.sub('\s\s+', ' ', feature) for feature in features]


KeyboardInterrupt: 